In [45]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer 
import xgboost as xgb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from joblib import dump, load
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, plot_confusion_matrix, classification_report
from sklearn.metrics import balanced_accuracy_score, accuracy_score, precision_score, recall_score, f1_score,roc_auc_score
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import RFE
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder,OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import balanced_accuracy_score, accuracy_score, precision_score, recall_score, f1_score,roc_auc_score
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectKBest
import scipy.stats as stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn.naive_bayes import GaussianNB
import shap

In [46]:
pd.set_option('display.max_rows', None)
df = pd.read_excel("Cardiac Rehab.xlsx")
df.head()

,Patient_Registration_Number,Race,Education_level,Patient_occupation,Health_funding,CR_Intake,Prescribed_Sessions,AACVPR_Risk_Category,CR_Adherence,Triglyceride,...,Duration_Between_Ward_Enrollment,Duration_CR,Pre_BMI,Post_BMI,Pre_BMI_range,Post_BMI_range,Pre_Peak_Heart_Rate_range,Post_Peak_Heart_Rate_range,Pre_METs_range,Post_Peak_METs_range
0,8444773,Others,Form 6 / pre-university graduate,Government servant,Fully Funded,Yes,8 weeks,Low,Yes,1.0,...,114.0,126.0,19.027399,19.788495,Normal,Normal,Moderate,NaN,Vigorous Intensity,NaN
1,15302617,Chinese,Unknown,Self-employed (excludes housewives),Self funded,Yes,8 weeks,Intermediate,Yes,2.3,...,116.0,123.0,35.751278,35.049617,Obesity,Obesity,NaN,NaN,Vigorous Intensity,Vigorous Intensity
2,6862705,Others,College/university graduate,Self-employed (excludes housewives),Self funded,Yes,8 weeks,Intermediate,Yes,1.2,...,151.0,92.0,23.610200,23.224413,Overweight,Overweight,NaN,NaN,Moderate Intensity,NaN
3,2282933,Malay,Some secondary education,Government servant,Fully Funded,Yes,8 weeks,Intermediate,Yes,1.2,...,88.0,69.0,18.552876,19.817844,Normal,Normal,NaN,NaN,Vigorous Intensity,Vigorous Intensity
4,3531899,Malay,Unknown,Government servant,Fully Funded,Yes,8 weeks,Low,Yes,1.2,...,29.0,126.0,27.636054,27.352608,Obesity,Overweight,Hard,NaN,Vigorous Intensity,NaN


In [47]:
#drop irrelevant column
drop_column = ['Patient_Registration_Number',
               'Patient_occupation',
               'Education_level',
                 'Health_funding',
                 'Residential_Postcode',
                 'postal_code',
                 'latitude',
                 'longitude',
                 'place_name',
                 'state_name',
              'CR_Intake']
df = df.drop(drop_column,axis=1).copy()

In [48]:
#drop redundant column
red_col = ['Triglyceride_cat', 'HDL_cat','LDL_cat','HbA1c_cat','Pre_BP_cat','CR_BP_cat','Pre_BMI_range','Post_BMI_range','Pre_Peak_Heart_Rate_range',
          'Pre_METs_range','Post_Peak_METs_range','Post_Peak_Heart_Rate_range','Exercise_frequency_sessions_week']  
df = df.drop(red_col, axis= 1).copy()

In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 184 entries, 0 to 183
Data columns (total 79 columns):
 #   Column                                               Non-Null Count  Dtype  
---  ------                                               --------------  -----  
 0   Race                                                 184 non-null    object 
 1   Prescribed_Sessions                                  184 non-null    object 
 2   AACVPR_Risk_Category                                 184 non-null    object 
 3   CR_Adherence                                         184 non-null    object 
 4   Triglyceride                                         181 non-null    float64
 5   HDL                                                  180 non-null    float64
 6   LDL                                                  180 non-null    float64
 7   HbA1c                                                167 non-null    float64
 8   Pre_Tobacco                                          184 non-null    o

In [50]:
#There is an outlier that I cant deal with
outlier_value = 702
df['Pre_BP_diastolic'].replace(outlier_value, np.nan, inplace=True)

In [51]:
#check missing values
missing_values = df.isna().sum()
print("Missing values:\n", missing_values)

Missing values:
 Race                                                    0
Prescribed_Sessions                                     0
AACVPR_Risk_Category                                    0
CR_Adherence                                            0
Triglyceride                                            3
HDL                                                     4
LDL                                                     4
HbA1c                                                  17
Pre_Tobacco                                             0
Post_Tobacco                                            2
Pre_Weight                                             13
Post_Weight                                            26
Height                                                  8
Pre_Exercise_Stress_Test                                0
Pre_Peak_Heart_Rate                                    40
Pre_METs                                                9
Post_Exercise_Stress_Test                              

In [52]:
# Calculate the percentage of missing values for each column
missing_percentage = df.isnull().mean() * 100
print(missing_percentage)
# Identify columns where this percentage is greater than 30%
cols_to_drop = missing_percentage[missing_percentage > 30].index
print(cols_to_drop)
# Drop these columns
df.drop(cols_to_drop, axis=1, inplace=True)

Race                                                    0.000000
Prescribed_Sessions                                     0.000000
AACVPR_Risk_Category                                    0.000000
CR_Adherence                                            0.000000
Triglyceride                                            1.630435
HDL                                                     2.173913
LDL                                                     2.173913
HbA1c                                                   9.239130
Pre_Tobacco                                             0.000000
Post_Tobacco                                            1.086957
Pre_Weight                                              7.065217
Post_Weight                                            14.130435
Height                                                  4.347826
Pre_Exercise_Stress_Test                                0.000000
Pre_Peak_Heart_Rate                                    21.739130
Pre_METs                 

In [53]:
#Impute missing value with mean
mean_col = ['HDL', 'LDL', 'Pre_Weight', 'Post_Weight', 'Height', 'Pre_Peak_Heart_Rate', 'Pre_METs',
            'Age','HbA1c']
mean_imputer = SimpleImputer(strategy='mean')

# Impute missing values in each column separately
df[mean_col] = pd.DataFrame(mean_imputer.fit_transform(df[mean_col]), columns=mean_col)

In [54]:
mode_col = ['Post_Tobacco','Post_Exercise_Stress_Test','MACCE','Pre_Left_Ventricle_EF']
mode_imputer = SimpleImputer(strategy='most_frequent')


df[mode_col] = pd.DataFrame(mode_imputer.fit_transform(df[mode_col]), columns=mode_col)

In [55]:
median_col = ['Triglyceride','Exercise_frequency_mins_week','Duration_Between_Ward_Enrollment','Duration_CR',
             'Pre_BMI','Post_BMI', 'CR_BP_systolic','CR_BP_diastolic','Pre_BP_diastolic','Pre_BP_systolic']
median_imputer = SimpleImputer(strategy='median')


df[median_col] = pd.DataFrame(median_imputer.fit_transform(df[median_col]), columns=median_col)

In [56]:
df = df[df["AACVPR_Risk_Category"] != "Did Not Enroll CR"]
df = df.reset_index(drop=True)

In [57]:
df[df["AACVPR_Risk_Category"] == "Unknown"]
df = df[df["AACVPR_Risk_Category"] != "Unknown"]
df = df.reset_index(drop=True)

In [58]:
missing_values = df.isna().sum()
print("Missing values:\n", missing_values)

Missing values:
 Race                                                   0
Prescribed_Sessions                                    0
AACVPR_Risk_Category                                   0
CR_Adherence                                           0
Triglyceride                                           0
HDL                                                    0
LDL                                                    0
HbA1c                                                  0
Pre_Tobacco                                            0
Post_Tobacco                                           0
Pre_Weight                                             0
Post_Weight                                            0
Height                                                 0
Pre_Exercise_Stress_Test                               0
Pre_Peak_Heart_Rate                                    0
Pre_METs                                               0
Post_Exercise_Stress_Test                              0
Exercise_frequ

In [59]:
# Analyzing the unique values in each column to determine if they are categorical or numerical
unique_values = df.nunique()

# Displaying the unique values for each column
unique_values.sort_values()

Admission_Diagnosis_Valve replacement(s)                 2
Admission_Diagnosis_Angina                               2
Admission_Diagnosis_Arrhythmia                           2
Admission_Diagnosis_CABG                                 2
Admission_Diagnosis_Heart Failure                        2
Admission_Diagnosis_NSTEMI                               2
Admission_Diagnosis_Other cardiothoracic procedures      2
Admission_Diagnosis_PCI                                  2
Admission_Diagnosis_STEMI                                2
Pre_Medication_Aspirin                                   2
Risk_Factors_DM Type 2                                   2
Gender                                                   2
Risk_Factors_High Lipid Profile                          2
Risk_Factors_None of the above                           2
Risk_Factors_Unknown                                     2
Pre_Medication_ACEI/ARB                                  2
Past_CV_Chronic AF                                      

In [60]:
selected_columns = unique_values[(unique_values >= 3) & (unique_values <= 5)].index.tolist()
selected_columns

['Race',
 'Prescribed_Sessions',
 'AACVPR_Risk_Category',
 'Pre_Tobacco',
 'Post_Tobacco',
 'Pre_Exercise_Stress_Test',
 'Post_Exercise_Stress_Test',
 'Pre_Left_Ventricle_EF',
 'Total_Risk_Factors']

In [61]:
categories = [['less then 40%', 'between 40 to 50%', 'more then 50% with no failure symptoms']]
encoder = OrdinalEncoder(categories=categories)
df["Pre_Left_Ventricle_EF"] = encoder.fit_transform(df[["Pre_Left_Ventricle_EF"]])
df['Pre_Left_Ventricle_EF']

0      2.0
1      2.0
2      2.0
3      0.0
4      2.0
5      2.0
6      2.0
7      0.0
8      2.0
9      2.0
10     2.0
11     0.0
12     2.0
13     2.0
14     0.0
15     2.0
16     0.0
17     2.0
18     2.0
19     2.0
20     2.0
21     1.0
22     2.0
23     2.0
24     2.0
25     2.0
26     2.0
27     1.0
28     0.0
29     1.0
30     2.0
31     2.0
32     2.0
33     1.0
34     2.0
35     2.0
36     1.0
37     2.0
38     2.0
39     0.0
40     1.0
41     2.0
42     1.0
43     1.0
44     1.0
45     2.0
46     2.0
47     2.0
48     2.0
49     0.0
50     2.0
51     1.0
52     0.0
53     2.0
54     2.0
55     2.0
56     2.0
57     2.0
58     0.0
59     2.0
60     2.0
61     2.0
62     1.0
63     2.0
64     2.0
65     2.0
66     2.0
67     2.0
68     2.0
69     2.0
70     0.0
71     2.0
72     2.0
73     1.0
74     0.0
75     2.0
76     0.0
77     2.0
78     1.0
79     2.0
80     1.0
81     2.0
82     0.0
83     1.0
84     0.0
85     2.0
86     2.0
87     2.0
88     0.0
89     2.0
90     2.0

In [62]:
categories = [['6 weeks', '8 weeks', '> 8 weeks']]
encoder = OrdinalEncoder(categories = categories)
df['Prescribed_Sessions'] = encoder.fit_transform(df[['Prescribed_Sessions']])

In [63]:
df['CR_Completion'] = df['CR_Completion'].map({'Yes': 1, 'No': 0})
df['CR_Completion'].value_counts()

1    155
0     18
Name: CR_Completion, dtype: int64

In [64]:
df['CR_Adherence'] = df['CR_Adherence'].map({'Yes': 1, 'No': 0})
df['CR_Adherence'].value_counts()

1    136
0     37
Name: CR_Adherence, dtype: int64

In [65]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 173 entries, 0 to 172
Data columns (total 73 columns):
 #   Column                                               Non-Null Count  Dtype  
---  ------                                               --------------  -----  
 0   Race                                                 173 non-null    object 
 1   Prescribed_Sessions                                  173 non-null    float64
 2   AACVPR_Risk_Category                                 173 non-null    object 
 3   CR_Adherence                                         173 non-null    int64  
 4   Triglyceride                                         173 non-null    float64
 5   HDL                                                  173 non-null    float64
 6   LDL                                                  173 non-null    float64
 7   HbA1c                                                173 non-null    float64
 8   Pre_Tobacco                                          173 non-null    o

In [66]:
cats = df.select_dtypes(exclude=np.number).columns.tolist()
print(cats)

['Race', 'AACVPR_Risk_Category', 'Pre_Tobacco', 'Post_Tobacco', 'Pre_Exercise_Stress_Test', 'Post_Exercise_Stress_Test', 'MACCE', 'Gender']


In [69]:
df['MACCE'] = df['MACCE'].astype('int64')


In [71]:
y = df['AACVPR_Risk_Category']
categories = [['Low', 'Intermediate', 'High']]

encoder = OrdinalEncoder(categories=categories)


df["AACVPR_Risk_Category"] = encoder.fit_transform(df[["AACVPR_Risk_Category"]])


y = df["AACVPR_Risk_Category"]
print(y.head())

0    0.0
1    1.0
2    1.0
3    1.0
4    0.0
Name: AACVPR_Risk_Category, dtype: float64


In [72]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 173 entries, 0 to 172
Data columns (total 73 columns):
 #   Column                                               Non-Null Count  Dtype  
---  ------                                               --------------  -----  
 0   Race                                                 173 non-null    object 
 1   Prescribed_Sessions                                  173 non-null    float64
 2   AACVPR_Risk_Category                                 173 non-null    float64
 3   CR_Adherence                                         173 non-null    int64  
 4   Triglyceride                                         173 non-null    float64
 5   HDL                                                  173 non-null    float64
 6   LDL                                                  173 non-null    float64
 7   HbA1c                                                173 non-null    float64
 8   Pre_Tobacco                                          173 non-null    o

In [73]:
#make categorical
cats = df.select_dtypes(exclude=np.number).columns.tolist()
for col in cats:
   df[col] = df[col].astype('category')

print(cats)

['Race', 'Pre_Tobacco', 'Post_Tobacco', 'Pre_Exercise_Stress_Test', 'Post_Exercise_Stress_Test', 'Gender']


In [74]:
df = pd.get_dummies(df, columns=cats, prefix=cats)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 173 entries, 0 to 172
Data columns (total 85 columns):
 #   Column                                               Non-Null Count  Dtype  
---  ------                                               --------------  -----  
 0   Prescribed_Sessions                                  173 non-null    float64
 1   AACVPR_Risk_Category                                 173 non-null    float64
 2   CR_Adherence                                         173 non-null    int64  
 3   Triglyceride                                         173 non-null    float64
 4   HDL                                                  173 non-null    float64
 5   LDL                                                  173 non-null    float64
 6   HbA1c                                                173 non-null    float64
 7   Pre_Weight                                           173 non-null    float64
 8   Post_Weight                                          173 non-null    f

In [75]:
df.to_csv('cardiac_clean16.csv', index=False)